In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io as skio
from skimage.transform import resize
import pandas as pd
import os
import requests
from pyarrow import feather
import io
import concurrent.futures

import warnings
warnings.filterwarnings("ignore")

In [2]:
def convert_audio_to_mel_image(filepath_to_audio, filepath_to_save, image_size=(128,512), n_mels=128, fmax=8000,):
    signal, sr = librosa.load(filepath_to_audio)
    mels = librosa.power_to_db(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels, fmax=fmax), ref=np.max)
    image = (((80+mels)/80)*255)
    image = np.flip(image, axis=0)
    resize_image = resize(image, image_size).astype(np.uint8)
    skio.imsave(filepath_to_save, resize_image)
    
def convert_audio_to_composite_image(filepath_to_audio, filepath_to_save, image_size=(128,512), n_mels=128, fmax=8000,):
    
    signal, sr = librosa.load(filepath_to_audio)
    
    mels = librosa.power_to_db(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels, fmax=fmax), ref=np.max)
    mel_image = (((80+mels)/80)*255)
    mel_image = np.flip(mel_image, axis=0)
    mel_image = resize(mel_image, (128,512)).astype(np.uint8)
    
    mfcc = librosa.power_to_db(librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=128, fmax=8000), ref=np.max)
    mfcc_image = (((80+mfcc)/80)*255)
    mfcc_image = np.flip(mfcc_image, axis=0)
    mfcc_image = resize(mfcc_image, (128,512)).astype(np.uint8)
    
    chromagram = librosa.feature.chroma_cqt(y=signal, sr=sr)
    chroma_image = resize(chromagram*255, (128,512)).astype(np.uint8)
    
    composite = np.dstack((mel_image, mfcc_image, chroma_image))
    
    skio.imsave(filepath_to_save, composite)

In [3]:
tracks_df = feather.read_feather('data/all_tracks.feather')

In [4]:
tracks_df = tracks_df.dropna(subset=['track_preview_link'])

In [5]:
random_tracks_df = tracks_df.sample(frac=1, random_state=42).reset_index(drop=True)
random_tracks_df

,track_id,track_name,track_preview_link,track_popularity,track_uri,release_date,artist_name,artist_id,artist_genres,artist_popularity
0,49xISBZpODXRwd7vVxGL93,Centipede,https://p.scdn.co/mp3-preview/1fc0bb6e8cb5bd83...,41,spotify:track:49xISBZpODXRwd7vVxGL93,2023-02-17,FIDLAR,3P6duIn7oHeiBACZfYeNud,"[indie garage rock, noise pop]",49
1,2dJrn376fJPUCj1f4txnRQ,Where Is My Mind?,https://p.scdn.co/mp3-preview/7f3f78ee83bfb672...,52,spotify:track:2dJrn376fJPUCj1f4txnRQ,2011-10-07,Trampled by Turtles,3GjVVVcFmUgEJEAAsbGkf4,"[duluth indie, indie folk, jam band, new ameri...",54
2,0tzixmHNQfE6S6SirSToxW,El Ultimo Adiós - Varios Artistas Version,https://p.scdn.co/mp3-preview/33af5b5bba9f9ca7...,73,spotify:track:0tzixmHNQfE6S6SirSToxW,2001-11-21,Alejandro Sanz,5sUrlPAHlS9NEirDB8SEbF,"[cantautor, latin arena pop, latin pop, mexica...",74
3,0u2AIKDVafHwCFQ9LDnqxH,Centipede,https://p.scdn.co/mp3-preview/a0ee99c887d09e54...,54,spotify:track:0u2AIKDVafHwCFQ9LDnqxH,2012,Knife Party,2DuJi13MWHjRHrqRUwk8vH,"[australian dance, brostep, complextro, edm, e...",52
4,034c3CHMQ2STRC6GPXXhl6,ニルバナ,https://p.scdn.co/mp3-preview/36dd3c1f6e6ff6de...,39,spotify:track:034c3CHMQ2STRC6GPXXhl6,2015-11-25,Tia,30rNCtpbgMaznRM9h8jlLv,[anime],41
...,...,...,...,...,...,...,...,...,...,...
35951,7qVnY9BiiGdIvXOgAvmTY9,Alright (Friendless Remix),https://p.scdn.co/mp3-preview/1f0269d9b30c42ef...,2,spotify:track:7qVnY9BiiGdIvXOgAvmTY9,2015-01-29,Fear Of Dawn,4yhf4iVtt3g7G8D43ZNhDU,[],6
35952,4hnLDkjxr9NPxwHcPArM9D,Resonant,https://p.scdn.co/mp3-preview/1ee3a2c186247874...,20,spotify:track:4hnLDkjxr9NPxwHcPArM9D,2018-05-04,Satoshi Tomiie,3TrAOZvW0MzZeKZRFnU7Ul,"[classic progressive house, japanese electroni...",26
35953,2Y0R8ey56JBZj6Au3HenCq,Máscara,https://p.scdn.co/mp3-preview/674acc795177ba16...,58,spotify:track:2Y0R8ey56JBZj6Au3HenCq,2003-04-16,Pitty,2dmQ0vMD3THLMcz7DsvfaT,"[brazilian rock, hard rock brasileiro, pop roc...",62
35954,3T3XVQvaJ09MckNdDE8hA8,Good Morning,https://p.scdn.co/mp3-preview/5d0b443bbbacab86...,41,spotify:track:3T3XVQvaJ09MckNdDE8hA8,2012-12-06,Brymo,094nOQ29vLC8FjZ3PhnM2u,"[afrobeats, afropop, alte, nigerian hip hop, n...",46


In [6]:
def download_preview_with_index(index):
    track_url = random_tracks_df.loc[index, 'track_preview_link']

    preview = requests.get(track_url)
    
    track_id = random_tracks_df.loc[index, 'track_id']

    filename = f'data/Spotify/mp3s/{track_id}.mp3'
    png_name = f'data/Spotify/comp_pngs/{track_id}.png'

    with open(filename, 'wb') as f:
        f.write(preview.content)

    convert_audio_to_composite_image(filename, png_name)

    os.remove(filename)
    
    print(index, end='\r')
    

In [ ]:
pd.options.mode.chained_assignment = None

indices = tracks_df.index
MAX_THREADS = 50
threads = min(MAX_THREADS, len(indices))
count = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_preview_with_index, indices)